In [1]:
from transformers import RobertaForSequenceClassification, AutoTokenizer,AutoModel,RobertaTokenizer

c:\users\data\appdata\local\programs\python\python38\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
model = RobertaForSequenceClassification.from_pretrained('klue/roberta-large',num_labels= 128)
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'cl

In [3]:
import pandas as pd
all_df = pd.read_csv('./input/train.csv')

In [10]:
from torch.utils.data import DataLoader , Dataset

In [58]:
class TourDataset(Dataset):
    def __init__(self, text, tokenizer , cats3, max_len,labels):
        super(TourDataset, self).__init__()
        self.text = text
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.cats3 = cats3
        self.labels = labels
    def __len__(self):
        return len(self.text)
    def __getitem__(self, item):
        text = self.text[item]
        cats3 =self.cats3[item]
        labels = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding = 'max_length',
            truncation = True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # return { 'input_ids' : encoding['input_ids'] , 'attention_mask' : encoding['attention_mask'],
        #         },cats3
        return text,labels

In [12]:
max_len = 225
batch_size = 1

In [11]:
all_df['cat3'] = all_df['cat3'].astype('category').cat.codes

In [59]:
def create_data_loader(df,tokenizer, max_len,batch_size  ,labels,shuffle_=False):
    ds = TourDataset(df.overview.to_numpy() ,
                    tokenizer,
                    df.cat3.to_numpy(),
                    max_len,
                     labels)

    return DataLoader(ds ,batch_size=batch_size, num_workers=0 , shuffle=shuffle_)

In [50]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
oh = OneHotEncoder()
cat3_oh = oh.fit_transform(np.reshape(all_df['cat3'].values,newshape=(-1,1)))

In [51]:
labels = cat3_oh.toarray()

In [60]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [61]:
train_data_loader = create_data_loader(all_df,tokenizer,max_len,batch_size,labels)

In [62]:
import torch.nn as nn

In [54]:
loss_fn = nn.CrossEntropyLoss()

In [27]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [19]:
tour_model = TourClassifier(text_model_name='klue/roberta-large')

In [20]:
#tok = tokenizer("3대에 걸쳐 아귀만을 전문으로 취급하는 전통과 역사를 자랑하는 음식점이다", return_tensors='pt')

In [21]:
#tok

In [22]:
#tour_model(**tok)

In [23]:
#model(**tok)
#model.train()

In [24]:
#labels = torch.tensor(label)
#outputs = model(sample, labels=labels)

In [ ]:
# 상속 구현을 안해서 ?

In [64]:
class TourClassifier(nn.Module):
  def __init__(self, text_model_name):
    super(TourClassifier, self).__init__()
    self.text_model = RobertaForSequenceClassification.from_pretrained(text_model_name)
    # self.text_model.gradient_checkpointing_enable()
    # self.drop = nn.Dropout(p=0.1)
    # def get_cls(target_size):
    #   return nn.Sequential(
    #       nn.Linear(self.text_model.config.hidden_size, self.text_model.config.hidden_size),
    #       nn.LayerNorm(self.text_model.config.hidden_size),
    #       nn.Dropout(p = 0.1),
    #       nn.ReLU(),
    #       nn.Linear(self.text_model.config.hidden_size, target_size),
    #   )
    # self.cls = get_cls(n_classes1)
    # self.cls2 = get_cls(n_classes2)
    # self.cls3 = get_cls(n_classes3)

  def forward(self, input_ids,attention_mask,label):
    text_output = self.text_model(input_ids=input_ids,attention_mask=attention_mask,labels=label)
    # image_output = self.image_model(pixel_values = pixel_values)
    # concat_outputs = torch.cat([text_output.last_hidden_state, image_output.last_hidden_state],1)
    #config hidden size 일치해야함
    # encoder_layer = nn.TransformerEncoderLayer(d_model=self.text_model.config.hidden_size, nhead=8)
    # transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)

    # outputs = transformer_encoder(text_output.last_hidden_state)
    #cls token
    # outputs = text_output.last_hidden_state[:,0]
    # output = self.drop(outputs)

    # out1 = self.cls(output)
    # out2 = self.cls2(output)
    # out3 = self.cls3(output)
    return text_output


In [65]:
model = TourClassifier('klue/roberta-large')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'class

In [66]:
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr= 25e-2)
loss_fn = nn.CrossEntropyLoss()
EPOCHS = 5
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_cosine_schedule_with_warmup(
optimizer,
num_warmup_steps=int(total_steps*0.1),
num_training_steps=total_steps
)

In [41]:
from tqdm import tqdm

In [28]:


import torch

In [ ]:
accuracies = AverageMeter()
f1_accuracies = AverageMeter()
model = RobertaForSequenceClassification.from_pretrained('klue/roberta-large')


In [ ]:
model.train()

In [42]:
def calc_tour_acc(pred, label):
    _, idx = pred.max(1)
    acc = torch.eq(idx, label).sum().item() / idx.size()[0]
    x = label.cpu().numpy()
    y = idx.cpu().numpy()
    f1_acc = f1_score(x, y, average='weighted')
    return acc,f1_acc


In [33]:
from sklearn.metrics import f1_score

In [34]:
from torch.optim import Adam

In [35]:
# optimizer = Adam(model.parameters(), lr=1e-6)
optimizer = Adam(model.parameters(), lr=25e-2)
itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0
model.train()

TourClassifier(
  (text_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(32000, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (

In [43]:
import torch.nn.functional as F

In [37]:
for t, l in train_data_loader:
    break

torch.Size([1, 128])

In [41]:
l[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.], dtype=torch.float64)

In [30]:

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

In [31]:
import torch

In [32]:
l

tensor([120], dtype=torch.int16)

In [43]:
labels

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.], dtype=torch.float64)

In [52]:
for param in model.parameters():
    print(param)
    break

Parameter containing:
tensor([[-2.4841e-02,  9.0714e-03, -1.7334e-02,  ...,  8.4900e-02,
         -6.3057e-03, -1.1841e-02],
        [-2.1957e-02,  2.0233e-02, -6.1340e-02,  ...,  5.7220e-03,
          1.5762e-02, -2.9282e-02],
        [-4.9835e-02, -3.4153e-05,  3.8330e-02,  ..., -4.5502e-02,
          4.6997e-02, -6.9336e-02],
        ...,
        [-3.2074e-02,  1.2573e-02,  2.1057e-02,  ...,  1.0730e-01,
          3.8910e-03, -1.2604e-02],
        [-5.2917e-02,  1.7151e-02, -1.5160e-02,  ...,  1.0449e-01,
          1.5823e-02, -4.1412e-02],
        [-1.0818e-02,  1.3573e-02,  1.0429e-02,  ...,  1.1719e-01,
         -1.0328e-03, -8.3694e-03]], requires_grad=True)


In [49]:
for param in model.parameters():
    print(param)
    break

Parameter containing:
tensor([[-2.4841e-02,  9.0714e-03, -1.7334e-02,  ...,  8.4900e-02,
         -6.3057e-03, -1.1841e-02],
        [-2.1957e-02,  2.0233e-02, -6.1340e-02,  ...,  5.7220e-03,
          1.5762e-02, -2.9282e-02],
        [-4.9835e-02, -3.4153e-05,  3.8330e-02,  ..., -4.5502e-02,
          4.6997e-02, -6.9336e-02],
        ...,
        [-3.2074e-02,  1.2573e-02,  2.1057e-02,  ...,  1.0730e-01,
          3.8910e-03, -1.2604e-02],
        [-5.2917e-02,  1.7151e-02, -1.5160e-02,  ...,  1.0449e-01,
          1.5823e-02, -4.1412e-02],
        [-1.0818e-02,  1.3573e-02,  1.0429e-02,  ...,  1.1719e-01,
         -1.0328e-03, -8.3694e-03]], requires_grad=True)


In [55]:
for param in model.parameters():
    print(param)
    break

Parameter containing:
tensor([[ 0.2252,  0.2591, -0.2673,  ...,  0.3349, -0.2563,  0.2382],
        [-0.0220,  0.0202, -0.0613,  ...,  0.0057,  0.0158, -0.0293],
        [-0.2998,  0.2500, -0.2117,  ...,  0.2045, -0.2030, -0.3193],
        ...,
        [-0.0321,  0.0126,  0.0211,  ...,  0.1073,  0.0039, -0.0126],
        [-0.0529,  0.0172, -0.0152,  ...,  0.1045,  0.0158, -0.0414],
        [-0.0108,  0.0136,  0.0104,  ...,  0.1172, -0.0010, -0.0084]],
       requires_grad=True)


In [57]:
for param in model.parameters():
    print(param)
    break

Parameter containing:
tensor([[ 0.3927,  0.4266, -0.4348,  ...,  0.5024, -0.4238,  0.4057],
        [-0.0220,  0.0202, -0.0613,  ...,  0.0057,  0.0158, -0.0293],
        [-0.4673,  0.4175, -0.3792,  ...,  0.3720, -0.3705, -0.4868],
        ...,
        [-0.0321,  0.0126,  0.0211,  ...,  0.1073,  0.0039, -0.0126],
        [-0.0529,  0.0172, -0.0152,  ...,  0.1045,  0.0158, -0.0414],
        [-0.0108,  0.0136,  0.0104,  ...,  0.1172, -0.0010, -0.0084]],
       requires_grad=True)


In [58]:
p_itr= 1

In [45]:
text

('소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다. 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다. 또한, 섬에는 각시여 전설, 도둑바위 등의 설화가 전해 내려오고 있으며, 매년 정월 풍어제 풍속이 이어지고 있다.<br>',)

In [47]:
import torch

('항구/포구',)

('항구/포구',)

In [ ]:
model.train()
for epoch in range(epochs):
    for text, label in train_data_loader:
        optimizer.zero_grad()
        # encoding and zero padding
        # encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        # padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]

        # sample = torch.tensor(padded_list)
        # sample, label = sample.to(device), label.to(device)
        encoding = tokenizer.encode_plus(
          text[0],
          add_special_tokens=True,
          max_length=max_len,
          return_token_type_ids=False,
          padding = 'max_length',
          truncation = True,
          return_attention_mask=True,
          return_tensors='pt',
        )
        labels = torch.tensor(label)
        #학습 샘플의 인풋이 (batch_size, sequence_length)로 들어간다는 것이다. 따라서 zero-padding을 직접 해줘서 model의 forward에 넣어줘야한다.
        # 긴 텍스트가 나올 때 자르기 필요 -> tokenizing 할떄 max_length padding
        # forward
        outputs = model(input_ids = encoding['input_ids'],attention_mask=encoding['attention_mask'], labels=labels)
        loss, logits = outputs['loss'] , outputs['logits']

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1
    break

C:\Users\DATA\AppData\Local\Temp\ipykernel_15372\1738767311.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
C:\Users\DATA\AppData\Local\Temp\ipykernel_15372\1738767311.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


In [61]:
sample.shape

torch.Size([1, 1316])

In [62]:
labels

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0.]], dtype=torch.float64)

In [ ]:
for param in model.parameters():
    break

In [ ]:
text = t

In [ ]:
ncoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]

In [ ]:
len(ncoded_list[0])

In [ ]:
#!git clone https://github.com/e9t/nsmc.git

In [64]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [65]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

In [66]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [67]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=1, shuffle=True, num_workers=0)

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification,RobertaForSequenceClassification

c:\users\data\appdata\local\programs\python\python38\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# 모델 구조와 관계있지 num_labels 과 input과는 관련없다 , num_labels가 128이라고 128 one-hot target을 target input으로 주는 게 아님
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=128)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [70]:
# optimizer = Adam(model.parameters(), lr=25e-2)
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

In [71]:
for text, label in train_loader:
    break

In [72]:
text

('쓰레기는 아닐지라도 결코 졸작을 면할길이 없는 영화를 찍는 방법.',)

In [73]:
label

tensor([0])

In [ ]:
for epoch in range(epochs):
    for text, label in train_loader:
        optimizer.zero_grad()

        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]

        sample = torch.tensor(padded_list)
        # sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs['loss'],outputs['logits']

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % 1 == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1
        # 똑같은 예제를 해도 batch가 1이기도 하고 너무느리다


C:\Users\DATA\AppData\Local\Temp\ipykernel_17172\3664207432.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
C:\Users\DATA\AppData\Local\Temp\ipykernel_17172\3664207432.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


[Epoch 1/1] Iteration 1 -> Train Loss: 0.0096, Accuracy: 0.000
[Epoch 1/1] Iteration 2 -> Train Loss: 0.0099, Accuracy: 0.000
[Epoch 1/1] Iteration 3 -> Train Loss: 0.0090, Accuracy: 0.000
[Epoch 1/1] Iteration 4 -> Train Loss: 0.0096, Accuracy: 0.000
[Epoch 1/1] Iteration 5 -> Train Loss: 0.0097, Accuracy: 0.000
[Epoch 1/1] Iteration 6 -> Train Loss: 0.0099, Accuracy: 0.000
[Epoch 1/1] Iteration 7 -> Train Loss: 0.0094, Accuracy: 0.000
[Epoch 1/1] Iteration 8 -> Train Loss: 0.0100, Accuracy: 0.000
[Epoch 1/1] Iteration 9 -> Train Loss: 0.0099, Accuracy: 0.000
[Epoch 1/1] Iteration 10 -> Train Loss: 0.0093, Accuracy: 0.000
[Epoch 1/1] Iteration 11 -> Train Loss: 0.0100, Accuracy: 0.000
[Epoch 1/1] Iteration 12 -> Train Loss: 0.0101, Accuracy: 0.000
[Epoch 1/1] Iteration 13 -> Train Loss: 0.0092, Accuracy: 0.000
[Epoch 1/1] Iteration 14 -> Train Loss: 0.0100, Accuracy: 0.000
[Epoch 1/1] Iteration 15 -> Train Loss: 0.0092, Accuracy: 0.000
[Epoch 1/1] Iteration 16 -> Train Loss: 0.0098, A

In [ ]:
w

In [ ]:
for e,r in train_data_loader:
    break

In [ ]:
 e

NameError: name 'epochs' is not defined

In [ ]:
data

In [ ]:
for step, (data,label) in tqdm(enumerate(train_data_loader)):
    break

In [ ]:
data

In [ ]:
label

In [ ]:
for step, (data,label) in tqdm(enumerate(train_data_loader)):
    batch_size = data['input_ids'].size(0)
    # input_ids = data['input_ids']
    # attention_mask = data['attention_mask']
    cats3 = data['cat3']
    input_ids = data['input_ids'][0]
    attention_mask = data['attention_mask'][0]
    # outputs = text_model(input_ids,attention_mask)
    # sentence_embed = outputs.last_hidden_state[:,0]
    # output = nn.Dropout(sentence_embed)
    # encoder_layer = nn.TransformerEncoderLayer(d_model=text_model.config.hidden_size, nhead=8).to(device)
    # transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2).to(device)

    # outputs = transformer_encoder(sentence_embed)
    # outputs = transformer_encoder(concat_outputs)
    #cls token
    outputs3 = model(input_ids = input_ids, attention_mask=attention_mask)
    _, preds = torch.max(outputs3.logits, dim=1)
    cats3 = cats3.type(torch.LongTensor)

    # loss1 = loss_fn(outputs1, cats1)
    # loss2 = loss_fn(outputs2, cats2)
    loss3 = loss_fn(outputs3.logits, cats3)
    print(loss3)
    # loss = loss1 * 0.05 + loss2 * 0.1 + loss3 * 0.85
    # 모델 매개 변수에 대한 변화도 수집

    loss3.backward()
    nn.utils.clip_grad_norm_(tour_model.parameters(), max_norm=1.0)
    # 변화도에 따라 업데이트 매개변수 조정
    optimizer.step()
    # scheduler의 의미: Learning Rate Scheduler => learning rate를 조절한다.
    scheduler.step()
    optimizer.zero_grad()

    if step % 1 == 0 or step == (len(train_data_loader)-1):
                acc,f1_acc = calc_tour_acc(outputs3.logits, cats3)
                accuracies.update(acc, batch_size)
                f1_accuracies.update(f1_acc, batch_size)
                print('Epoch: [{0}][{1}/{2}] '
                      'Acc: {acc.val:.3f}({acc.avg:.3f}) '
                      'f1_Acc: {f1_acc.val:.3f}({f1_acc.avg:.3f}) '
                      .format(
                      EPOCHS, step+1, len(train_data_loader),
                      acc=accuracies,
                      f1_acc=f1_accuracies,
                      ))
    # out1 = self.cls(output)
    # out2 = self.cls2(output)
    # out3 = self.cls3(output)